In [33]:
import pandas as pd

In [34]:
matches = pd.read_csv("matches.csv", index_col = 0)

In [35]:
players = pd.read_csv("players.csv", index_col = 0)

In [36]:
82 * 30

2460

In [37]:
matches.shape

(2460, 14)

In [38]:
matches["Team"].value_counts()

Team
MIA    84
MEM    84
SAC    83
DAL    83
CHI    83
IND    82
TOR    82
BRK    82
ATL    82
ORL    82
PHI    82
DET    82
CLE    82
BOS    82
POR    82
SAS    82
CHO    82
WAS    82
MIN    82
LAL    82
LAC    82
DEN    82
NOP    82
UTA    82
GSW    82
PHO    82
NYK    81
MIL    80
OKC    80
HOU    80
Name: count, dtype: int64

In [39]:
matches.dtypes

Date           object
Start (ET)     object
Venue          object
Opponent       object
Result         object
Tm              int64
Opp             int64
W               int64
L               int64
Streak         object
Attend.       float64
LOG            object
Season          int64
Team           object
dtype: object

In [40]:
matches["Date"] = pd.to_datetime(matches["Date"])

In [41]:
matches["venue_code"] = matches["Venue"].astype("category").cat.codes

In [42]:
matches["opp_code"] = matches["Opponent"].astype("category").cat.codes

In [43]:
matches["hour"] = matches["Start (ET)"].str.replace(":.+", "", regex=True).astype("int")

In [44]:
matches["day_code"] = matches["Date"].dt.dayofweek

In [46]:
matches["target"] = (matches["Result"] == "W").astype("int")

In [47]:
matches

,Date,Start (ET),Venue,Opponent,Result,Tm,Opp,W,L,Streak,Attend.,LOG,Season,Team,venue_code,opp_code,hour,day_code,target
G,,,,,,,,,,,,,,,,,,,
1,2024-10-23,7:30p,Away,Toronto Raptors,W,136,106,1,0,W 1,19800.0,2:18,2024,CLE,0,27,7,2,1
2,2024-10-25,7:30p,Home,Detroit Pistons,W,113,101,2,0,W 2,19432.0,2:10,2024,CLE,1,8,7,4,1
3,2024-10-26,7:00p,Away,Washington Wizards,W,135,116,3,0,W 3,15156.0,2:23,2024,CLE,0,29,7,5,1
4,2024-10-28,7:30p,Away,New York Knicks,W,110,104,4,0,W 4,19812.0,2:23,2024,CLE,0,19,7,0,1
5,2024-10-30,7:00p,Home,Los Angeles Lakers,W,134,110,5,0,W 5,19432.0,2:07,2024,CLE,1,13,7,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,2025-04-04,7:00p,Away,Indiana Pacers,L,112,140,16,62,L 8,17274.0,2:13,2024,UTA,0,11,7,4,0
79,2025-04-06,6:00p,Away,Atlanta Hawks,L,134,147,16,63,L 9,16331.0,2:13,2024,UTA,0,0,6,6,0
80,2025-04-09,9:00p,Home,Portland Trail Blazers,W,133,126,17,63,W 1,18175.0,2:28,2024,UTA,1,24,9,2,1


In [51]:
from sklearn.ensemble import RandomForestClassifier as rfc

In [52]:
rf = rfc(n_estimators=50, min_samples_split=10, random_state=1)

In [54]:
train = matches[matches["Date"] < "2025-01-01"]

In [55]:
test = matches[matches["Date"] > "2025-01-01"]

In [59]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [60]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [61]:
preds = rf.predict(test[predictors])

In [62]:
from sklearn.metrics import accuracy_score as ac

In [63]:
acc = ac(test["target"], preds)

In [64]:
acc

0.5571236559139785

In [67]:
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [69]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,426,318
1,341,403


In [71]:
from sklearn.metrics import precision_score as pcs

In [72]:
pcs(test["target"], preds)

0.5589459084604715

In [74]:
grouped_matches = matches.groupby("Team")

In [75]:
group = grouped_matches.get_group("NYK")

In [76]:
group

,Date,Start (ET),Venue,Opponent,Result,Tm,Opp,W,L,Streak,Attend.,LOG,Season,Team,venue_code,opp_code,hour,day_code,target
G,,,,,,,,,,,,,,,,,,,
1,2024-10-22,7:30p,Away,Boston Celtics,L,109,132,0,1,L 1,19156.0,2:04,2024,NYK,0,1,7,1,0
2,2024-10-25,7:30p,Home,Indiana Pacers,W,123,98,1,1,W 1,19812.0,2:21,2024,NYK,1,11,7,4,1
3,2024-10-28,7:30p,Home,Cleveland Cavaliers,L,104,110,1,2,L 1,19812.0,2:23,2024,NYK,1,5,7,0,0
4,2024-10-30,7:30p,Away,Miami Heat,W,116,107,2,2,W 1,19620.0,2:10,2024,NYK,0,15,7,2,1
5,2024-11-01,7:00p,Away,Detroit Pistons,W,128,98,3,2,W 2,17022.0,1:55,2024,NYK,0,8,7,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,2025-04-06,7:00p,Home,Phoenix Suns,W,112,98,50,28,W 2,19812.0,2:14,2024,NYK,1,23,7,6,1
79,2025-04-08,7:30p,Home,Boston Celtics,L,117,119,50,29,L 1,19812.0,2:42,2024,NYK,1,1,7,1,0
80,2025-04-10,7:00p,Away,Detroit Pistons,L,106,115,50,30,L 2,20062.0,2:16,2024,NYK,0,8,7,3,0


In [81]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Date")
    rolling_stats = group[cols].rolling(3, closed = "left").mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [82]:
cols = ["Tm", "Opp"]
new_cols = [f"{c}_rolling" for c in cols]

In [83]:
rolling_averages(group, cols, new_cols)

,Date,Start (ET),Venue,Opponent,Result,Tm,Opp,W,L,Streak,...,LOG,Season,Team,venue_code,opp_code,hour,day_code,target,Tm_rolling,Opp_rolling
G,,,,,,,,,,,,,,,,,,,,,
4,2024-10-30,7:30p,Away,Miami Heat,W,116,107,2,2,W 1,...,2:10,2024,NYK,0,15,7,2,1,112.000000,113.333333
5,2024-11-01,7:00p,Away,Detroit Pistons,W,128,98,3,2,W 2,...,1:55,2024,NYK,0,8,7,4,1,114.333333,105.000000
6,2024-11-04,8:45p,Away,Houston Rockets,L,97,109,3,3,L 1,...,2:08,2024,NYK,0,10,8,0,0,116.000000,105.000000
7,2024-11-06,7:30p,Away,Atlanta Hawks,L,116,121,3,4,L 2,...,2:16,2024,NYK,0,0,7,2,0,113.666667,104.666667
8,2024-11-08,7:30p,Home,Milwaukee Bucks,W,116,94,4,4,W 1,...,2:07,2024,NYK,1,16,7,4,1,113.666667,109.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,2025-04-06,7:00p,Home,Phoenix Suns,W,112,98,50,28,W 2,...,2:14,2024,NYK,1,23,7,6,1,110.333333,106.666667
79,2025-04-08,7:30p,Home,Boston Celtics,L,117,119,50,29,L 1,...,2:42,2024,NYK,1,1,7,1,0,112.666667,109.000000
80,2025-04-10,7:00p,Away,Detroit Pistons,L,106,115,50,30,L 2,...,2:16,2024,NYK,0,8,7,3,0,116.666667,107.333333


In [137]:
matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols))

C:\Users\khull\AppData\Local\Temp\ipykernel_15444\578198639.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols))


In [138]:
matches_rolling = matches_rolling.droplevel("Team")

In [139]:
matches_rolling.index = range(matches_rolling.shape[0])

In [140]:
matches_rolling

,Date,Start (ET),Venue,Opponent,Result,Tm,Opp,W,L,Streak,...,LOG,Season,Team,venue_code,opp_code,hour,day_code,target,Tm_rolling,Opp_rolling
0,2024-10-28,7:30p,Home,Washington Wizards,L,119,121,2,2,L 2,...,2:26,2024,ATL,1,29,7,0,0,116.333333,121.333333
1,2024-10-30,7:00p,Away,Washington Wizards,L,120,133,2,3,L 3,...,2:25,2024,ATL,0,29,7,2,0,116.000000,123.000000
2,2024-11-01,7:30p,Home,Sacramento Kings,L,115,123,2,4,L 4,...,2:16,2024,ATL,1,25,7,4,0,114.333333,127.333333
3,2024-11-03,7:00p,Away,New Orleans Pelicans,W,126,111,3,4,W 1,...,2:07,2024,ATL,0,18,7,6,1,118.000000,125.666667
4,2024-11-04,7:45p,Home,Boston Celtics,L,93,123,3,5,L 1,...,1:56,2024,ATL,1,1,7,0,0,120.333333,122.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2365,2025-04-06,6:00p,Away,Boston Celtics,L,90,124,17,61,L 2,...,1:52,2024,WAS,0,1,6,6,0,102.333333,113.333333
2366,2025-04-08,7:00p,Away,Indiana Pacers,L,98,104,17,62,L 3,...,2:07,2024,WAS,0,11,7,1,0,101.000000,114.666667
2367,2025-04-09,7:00p,Home,Philadelphia 76ers,L,103,122,17,63,L 4,...,2:16,2024,WAS,1,22,7,2,0,95.000000,112.333333
2368,2025-04-11,8:00p,Away,Chicago Bulls,L,89,119,17,64,L 5,...,2:06,2024,WAS,0,4,8,4,0,97.000000,116.666667


In [217]:
def make_predictions(data, predictor):
    train = data[data["Date"] < "2025-01-01"]
    test = data[data["Date"] > "2025-01-01"]
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], prediction=preds), index=test.index)
    precision = pcs(test["target"], preds)
    return combined, precision

In [218]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [219]:
precision

0.5697050938337802

In [220]:
combined

,actual,prediction
29,0,0
30,0,0
31,1,1
32,0,1
33,1,1
...,...,...
2365,0,1
2366,0,0
2367,0,1
2368,0,1


In [221]:
combined = combined.merge(matches_rolling[["Date", "Team", "Opponent", "Result"]], left_index = True, right_index = True)

In [222]:
combined

,actual,prediction,Date,Team,Opponent,Result
29,0,0,2025-01-03,ATL,Los Angeles Lakers,L
30,0,0,2025-01-04,ATL,Los Angeles Clippers,L
31,1,1,2025-01-07,ATL,Utah Jazz,W
32,0,1,2025-01-09,ATL,Phoenix Suns,L
33,1,1,2025-01-14,ATL,Phoenix Suns,W
...,...,...,...,...,...,...
2365,0,1,2025-04-06,WAS,Boston Celtics,L
2366,0,0,2025-04-08,WAS,Indiana Pacers,L
2367,0,1,2025-04-09,WAS,Philadelphia 76ers,L
2368,0,1,2025-04-11,WAS,Chicago Bulls,L


In [223]:
class MissingDict(dict):
    __missing__ = lambda self, key : key

map_values = {
    "ATL": "Atlanta Hawks",
    "BOS": "Boston Celtics",
    "BRK": "Brooklyn Nets",
    "CHA": "Charlotte Hornets",
    "CHI": "Chicago Bulls",
    "CLE": "Cleveland Cavaliers",
    "DAL": "Dallas Mavericks",
    "DEN": "Denver Nuggets",
    "DET": "Detroit Pistons",
    "GSW": "Golden State Warriors",
    "HOU": "Houston Rockets",
    "IND": "Indiana Pacers",
    "LAC": "Los Angeles Clippers",
    "LAL": "Los Angeles Lakers",
    "MEM": "Memphis Grizzlies",
    "MIA": "Miami Heat",
    "MIL": "Milwaukee Bucks",
    "MIN": "Minnesota Timberwolves",
    "NOP": "New Orleans Pelicans",
    "NYK": "New York Knicks",
    "OKC": "Oklahoma City Thunder",
    "ORL": "Orlando Magic",
    "PHI": "Philadelphia 76ers",
    "PHX": "Phoenix Suns",
    "POR": "Portland Trail Blazers",
    "SAC": "Sacramento Kings",
    "SAS": "San Antonio Spurs",
    "TOR": "Toronto Raptors",
    "UTA": "Utah Jazz",
    "WAS": "Washington Wizards"
}
mapping = MissingDict(**map_values)

In [224]:
combined["new_team"] = combined["Team"].map(mapping)

In [225]:
combined

,actual,prediction,Date,Team,Opponent,Result,new_team
29,0,0,2025-01-03,ATL,Los Angeles Lakers,L,Atlanta Hawks
30,0,0,2025-01-04,ATL,Los Angeles Clippers,L,Atlanta Hawks
31,1,1,2025-01-07,ATL,Utah Jazz,W,Atlanta Hawks
32,0,1,2025-01-09,ATL,Phoenix Suns,L,Atlanta Hawks
33,1,1,2025-01-14,ATL,Phoenix Suns,W,Atlanta Hawks
...,...,...,...,...,...,...,...
2365,0,1,2025-04-06,WAS,Boston Celtics,L,Washington Wizards
2366,0,0,2025-04-08,WAS,Indiana Pacers,L,Washington Wizards
2367,0,1,2025-04-09,WAS,Philadelphia 76ers,L,Washington Wizards
2368,0,1,2025-04-11,WAS,Chicago Bulls,L,Washington Wizards


In [226]:
merged = combined.merge(
    combined,
    left_on=["Date", "Opponent"],
    right_on=["Date", "new_team"],
    suffixes=('', '_opponent'),
    how='inner'
)

In [216]:
merged

,actual,prediction,Date,Team,Opponent,Result,new_team,actual_opponent,prediction_opponent,Team_opponent,Opponent_opponent,Result_opponent,new_team_opponent
0,0,0,2025-01-03,ATL,Los Angeles Lakers,L,Atlanta Hawks,1,0,LAL,Atlanta Hawks,W,Los Angeles Lakers
1,0,0,2025-01-04,ATL,Los Angeles Clippers,L,Atlanta Hawks,1,0,LAC,Atlanta Hawks,W,Los Angeles Clippers
2,1,1,2025-01-07,ATL,Utah Jazz,W,Atlanta Hawks,0,0,UTA,Atlanta Hawks,L,Utah Jazz
3,1,0,2025-01-15,ATL,Chicago Bulls,W,Atlanta Hawks,0,0,CHI,Atlanta Hawks,L,Chicago Bulls
4,1,1,2025-01-18,ATL,Boston Celtics,W,Atlanta Hawks,0,0,BOS,Atlanta Hawks,L,Boston Celtics
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,0,1,2025-04-06,WAS,Boston Celtics,L,Washington Wizards,1,1,BOS,Washington Wizards,W,Boston Celtics
1384,0,0,2025-04-08,WAS,Indiana Pacers,L,Washington Wizards,1,1,IND,Washington Wizards,W,Indiana Pacers
1385,0,1,2025-04-09,WAS,Philadelphia 76ers,L,Washington Wizards,1,1,PHI,Washington Wizards,W,Philadelphia 76ers
1386,0,1,2025-04-11,WAS,Chicago Bulls,L,Washington Wizards,1,1,CHI,Washington Wizards,W,Chicago Bulls


In [227]:
merged[(merged["prediction"] == 1) &(merged["prediction_opponent"] == 0)]["actual"].value_counts()

actual
1    209
0    122
Name: count, dtype: int64

In [228]:
209/(209+122)

0.6314199395770392